## Первое ДЗ
**Результаты обучения:**
- **Начальный loss:** ≈ 4.66
- **Лучший val-loss:**  ≈ 1.8038
- **Финальный:** train ≈ 1.75, val ≈ 1.81


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size    = 32
block_size    = 64
max_iters     = 8000
eval_interval = 200
learning_rate = 5e-4
n_embd        = 128
n_head        = 8
n_layer       = 6
dropout       = 0.1
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters    = 300

torch.manual_seed(1337)

with open('nekrasov.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("Размер", len(text))
print(text[:1000])
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    d = train_data if split=='train' else val_data
    idx = torch.randint(len(d)-block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size]   for i in idx])
    y = torch.stack([d[i+1:i+block_size+1] for i in idx])
    return x.to(device), y.to(device)

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x); q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        return wei @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj  = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.GELU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa   = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1  = nn.LayerNorm(n_embd)
        self.ln2  = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb    = nn.Embedding(len(chars), n_embd)
        self.pos_emb      = nn.Embedding(block_size, n_embd)
        self.blocks       = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_final     = nn.LayerNorm(n_embd)
        self.head         = nn.Linear(n_embd, len(chars))
    def forward(self, idx, targets=None):
        B,T = idx.shape
        tok = self.token_emb(idx)
        pos = self.pos_emb(torch.arange(T, device=device))
        x = tok + pos
        x = self.blocks(x)
        x = self.ln_final(x)
        logits = self.head(x)
        if targets is None:
            return logits, None
        loss = F.cross_entropy(logits.view(-1, len(chars)), targets.view(-1))
        return logits, loss
    def generate(self, idx, max_new):
        for _ in range(max_new):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx

model = GPT().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters())/1e6:.3f}M")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train','val']:
        losses = []
        for _ in range(eval_iters):
            X,Y = get_batch(split)
            _, l = model(X,Y)
            losses.append(l.item())
        out[split] = sum(losses)/len(losses)
    model.train()
    return out

train_log, val_log = [], []
for it in range(max_iters):
    if it % eval_interval==0 or it==max_iters-1:
        los = estimate_loss()
        train_log.append((it, los['train']))
        val_log.append((it,   los['val']))
        print(f"step {it}: train {los['train']:.4f}, val {los['val']:.4f}")
    xb,yb = get_batch('train')
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


for prompt in ["Поставьте 20 баллов"]:
    ctx = torch.tensor([encode(prompt)], device=device)
    out = model.generate(ctx, max_new=200)[0].tolist()
    print(f"\n: {prompt}\n{decode(out)}\n")


Размер 1255871


Великих зрелищ, мировых судеб
Исконные, кровавые враги,
Пожар войны полмира обхватил,
И заревом зловещим осветились
Деяния держав миролюбивых...
Обращены в позорище вражды
Моря и суша... медленно и глухо
К нам двинулись громады кораблей,
Хвастливо предрекая нашу гибель,
И наконец приблизились - стоят
Пред укрепленной русскою твердыней...
И ныне в урне роковой лежат
Два жребия... и наступает время,
Когда Решитель мира и войны
Исторгнет их всесильною рукой
И свету потрясенному покажет.

----------------------------------------



Я покинул кладбище унылое,
Но я мысль мою там позабыл,-
Под землею в гробу приютилася
И глядит на тебя, мертвый друг!
Ты схоронен в морозы трескучие,
Жадный червь не коснулся тебя,
На лицо через щели гробовые
Проступить не успела вода
Ты лежишь, как сейчас похороненный,
Только словно длинней и белей
Пальцы рук, на груди твоей сложенных,
Да сквозь землю проникнувшим инеем
Убелил твои кудри мороз,
Да следы наложили чуть видные
Поцелуи суровой зимы